In [1]:
import numpy as np

def h(x,H):
    d = x-H
    d = np.multiply(d,d)
    d = np.sqrt(d.sum(axis=1))
    d_col_1 = d[1]-d[0]
    d_col_2 = d[2]-d[0] 
    d_col_3 = d[3]-d[0]
    di = np.array([d_col_1,d_col_2,d_col_3])
    di = di.T
    return di

class UGMM(object):
    '''Univariate GMM with CAVI'''
    def __init__(self, X, K=2, sigma=1):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]
        self.sigma2 = sigma**2

    def _init(self):
        self.phi = np.random.dirichlet([np.random.random()*np.random.randint(1, 10)]*self.K, self.N)
        print(self.phi)
        self.m = np.random.randint(int(self.X.min()), high=int(self.X.max()), size=self.K).astype(float)
        self.m += self.X.max()*np.random.random(self.K)
        self.s2 = np.ones(self.K) * np.random.random(self.K)
        print('Init mean')
        print(self.m)
        print('Init s2')
        print(self.s2)

    def get_elbo(self):
        t1 = np.log(self.s2) - self.m/self.sigma2
        t1 = t1.sum()
        t2 = -0.5*np.add.outer(self.X**2, self.s2+self.m**2)
        t2 += np.outer(self.X, self.m)
        t2 -= np.log(self.phi)
        t2 *= self.phi
        t2 = t2.sum()
        return t1 + t2

    def fit(self, max_iter=100, tol=1e-10):
        self._init()
        self.elbo_values = [self.get_elbo()]
        self.m_history = [self.m]
        self.s2_history = [self.s2]
        for iter_ in range(1, max_iter+1):
            self._cavi()
            self.m_history.append(self.m)
            self.s2_history.append(self.s2)
            self.elbo_values.append(self.get_elbo())
            if iter_ % 5 == 0:
                print(iter_, self.m_history[iter_])
            if np.abs(self.elbo_values[-2] - self.elbo_values[-1]) <= tol:
                print('ELBO converged with ll %.3f at iteration %d'%(self.elbo_values[-1],
                                                                     iter_))
                break

        if iter_ == max_iter:
            print('ELBO ended with ll %.3f'%(self.elbo_values[-1]))

    def _cavi(self):
        self._update_phi()
        self._update_mu()

    def _update_phi(self):
        t1 = np.outer(self.X, self.m)
        t2 = -(0.5*self.m**2 + 0.5*self.s2)
        exponent = t1 + t2[np.newaxis, :]
        self.phi = np.exp(exponent)
        self.phi = self.phi / self.phi.sum(1)[:, np.newaxis]

    def _update_mu(self):
        self.m = (self.phi*self.X[:, np.newaxis]).sum(0) * (1/self.sigma2 + self.phi.sum(0))**(-1)
        assert self.m.size == self.K
        #print(self.m)
        self.s2 = (1/self.sigma2 + self.phi.sum(0))**(-1)
        assert self.s2.size == self.K

import matplotlib.pyplot as plt
import seaborn as sns

bias = 0
std = 1

x = np.array([7.54,4.8])
H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
d = x-H
d = np.multiply(d,d)
d = np.sqrt(d.sum(axis=1))

d_col_1 = d[0]-d[1]
d_col_2 = d[2]-d[1] 
d_col_3 = d[3]-d[1]
real_arr = np.array([d_col_1,d_col_2,d_col_3])
print(real_arr)

SAMPLE = 100
from openpyxl import load_workbook
#data_only=Ture로 해줘야 수식이 아닌 값으로 받아온다.
load_wb = load_workbook("/Users/swbaek/Downloads/TDOA.xlsx", data_only=True)
#시트 이름으로 불러오기
load_ws = load_wb['Sheet']
#셀 좌표로 값 출력
d21 = np.sqrt(pow(load_ws.cell(1,1).value/1000,2)-pow(1.2,2))
d31 = np.sqrt(pow(load_ws.cell(1,2).value/1000,2)-pow(1.2,2))
d41 = np.sqrt(pow(load_ws.cell(1,2).value/1000,2)-pow(1.2,2))
X = np.array([d21,d31,d41])

for i in range(2,SAMPLE+1):
    d21 = np.sqrt(pow(load_ws.cell(i,1).value/1000,2)-pow(1.2,2))
    d31 = np.sqrt(pow(load_ws.cell(i,2).value/1000,2)-pow(1.2,2))
    d41 = np.sqrt(pow(load_ws.cell(i,3).value/1000,2)-pow(1.2,2))
    X = np.append(X,[d21,d31,d41])
#X = np.random.normal(loc=mu_arr[0], scale=1, size=SAMPLE)
print(X)
#for i, mu in enumerate(mu_arr[1:]):
#    X = np.append(X, np.random.normal(loc=mu, scale=1, size=SAMPLE))

fig, ax = plt.subplots(figsize=(15, 4))
sns.distplot(X[:SAMPLE], ax=ax, rug=True)
sns.distplot(X[SAMPLE:SAMPLE*2], ax=ax, rug=True)
sns.distplot(X[SAMPLE*2:], ax=ax, rug=True)
import time
start = time.time()
ugmm = UGMM(X, 3)
ugmm.fit()
print("time :", time.time() - start)
ugmm.phi.argmax(1)
#sorted(mu_arr)
#sorted(ugmm.m)
#for i in range(0,3)
x_init = np.array([1,1])

fig, ax = plt.subplots(figsize=(15, 4))
plt.grid()
sns.distplot(X[:SAMPLE], ax=ax, hist=True, norm_hist=True)
sns.distplot(np.random.normal(ugmm.m[0], 1, SAMPLE), color='k', hist=False, kde=True)
sns.distplot(X[SAMPLE:SAMPLE*2], ax=ax, hist=True, norm_hist=True)
sns.distplot(np.random.normal(ugmm.m[1], 1, SAMPLE), color='k', hist=False, kde=True)
sns.distplot(X[SAMPLE*2:], ax=ax, hist=True, norm_hist=True)
sns.distplot(np.random.normal(ugmm.m[2], 1, SAMPLE), color='k', hist=False, kde=True)

print(ugmm.m_history)

[2.39       5.75088231 4.751592  ]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/swbaek/Downloads/TDOA.xlsx'

In [17]:
from scipy.stats import boltzmann
print(boltzmann.ppf(0,1:0.1:1,0.1,4)

SyntaxError: invalid syntax (<ipython-input-17-574e92680ce2>, line 2)